======================================================================
       Adquisición Multicanal de Datos en 4 Canales para Red Pitaya
======================================================================

Desarrollado por Ing. Mattenet Mariana 
Departamento de Telecomunicaciones
Instituto Balseiro - Comisión Nacional de Energía Atómica -
Año: 2025

Descripción:
------------
Este script permite la adquisición de datos en una Red Pitaya, capturando 
señales en múltiples canales de manera simultánea con un trigger basado en 
un umbral configurable. Los datos se almacenan en archivos HDF5 para una 
manipulación eficiente.

Funcionalidades principales:
----------------------------
- Selección de canales de adquisición (1 a 4).
- Configuración de parámetros de adquisición:
  * Nivel de trigger (V)
  * Número de muestras por evento
  * Delay de muestras
- Cálculo automático del número máximo de eventos según el espacio disponible en SD.
- Creación y manejo de archivos HDF5 para almacenar los datos de forma estructurada.
- División de archivos cuando alcanzan un umbral de tamaño.
- Configuración de la hora de adquisición (automática o manual).

Flujo del programa:
-------------------
1. Configuración del entorno y selección de la hora de adquisición.
2. Inicialización de la FPGA y configuración del sistema de adquisición.
3. Selección de canales y parámetros de adquisición.
4. Cálculo del número de eventos posibles según el espacio disponible.
5. Inicio de la adquisición:
    - Espera de eventos con trigger.
    - Captura de datos y almacenamiento en archivos HDF5.
    - Creación de nuevos archivos si se supera el tamaño umbral.
6. Finalización del proceso y liberación de la FPGA.

Formato de los archivos HDF5 generados:
----------------------------------------
- Nombre del archivo: `Data_YYMMDD_HHMM_XXXX.h5`
  (YYMMDD: año, mes, día; HHMM hora, minutos; XXXX: índice secuencial)
- Atributos generales:
  * Hora de inicio de adquisición
  * Tasa de muestreo
  * Nivel de trigger
  * Número de muestras por evento y delay
  * Canales utilizados
  * Cantidad total de eventos
- Estructura por evento:
  /event_000001/
    ├── channel_1 (array de muestras)
    ├── channel_2 (array de muestras)
    ├── channel_3 (array de muestras)
    ├── channel_4 (array de muestras)
    ├── timestamp (marca de tiempo del trigger)

Advertencias:
-------------
- Asegúrese de que la tarjeta SD tenga suficiente espacio libre (al menos 200 MB recomendados).
- Verifique que la Red Pitaya esté correctamente conectada y configurada antes de iniciar la adquisición.
- Si no se detectan triggers, el proceso puede quedar esperando eventos.
- Tenga en cuenta que el rango de tensión de las entradas de la Red Pitaya depende de la configuración del
jumper en la placa (HV: ±20 V, LV: ±1 V). Se recomienda verificar que las señales a adquirir estén dentro
del rango adecuado antes de comenzar la adquisición.

Para más información, consulte la documentación oficial:
https://redpitaya.readthedocs.io/en/latest/intro.html

Uso:
----
Simplemente ejecute el script en Python. Se le pedirá que ingrese los parámetros necesarios.

Para más información, consulte la documentación oficial de Red Pitaya:
https://redpitaya.readthedocs.io/en/latest/intro.html

Descripción:
Este programa realiza la adquisición de datos en una Red Pitaya, 
capturando señales en hasta 4 canales de manera simultánea con un trigger 
basado en un umbral en el canal 2 (opcional). Utiliza un buffer circular de 
tamaño 16,384 muestras y permite configurar parámetros como número de muestras 
por evento, retraso de muestras y cantidad de eventos a capturar.

Antes de iniciar la adquisición, se deben configurar los parametros de adquisicion 
deseados y seleccionar si la captura será continua o acotada al espacio libre en 
la SD de la RedPitaya. Si se elige esta última opción el programa verifica el 
espacio libre en la SD y se calcula el número máximo de eventos permitidos, asegurando 
al menos 200 MB libres. 

Los datos adquiridos se almacenan en formato HDF5, permitiendo optimización y 
manipulación eficiente de grandes volúmenes de datos. Si se desea trabajar con 
adquisición de grandes volumenes de datos se puede ejecutar el programa monitor_hdf5
que transmite los datos capturados a traves de la red y los borra de la memoria de la
RedPitaya para liberar espacio.

Nota: El rango de tensión de las entradas de la Red Pitaya depende de la 
configuración del jumper en la placa (HV: ±20 V, LV: ±1 V). Se recomienda 
verificar que las señales medidas estén dentro del rango adecuado antes 
de la adquisición. Para más información, consulte la documentación oficial:
https://redpitaya.readthedocs.io/en/latest/intro.html

==============================================================

# Descripción:
Este programa realiza la adquisición de datos en una Red Pitaya, capturando señales en hasta 4 canales de manera simultánea con un trigger basado en un umbral en el canal 2 (opcional). Utiliza un buffer circular de tamaño 16384 muestras y permite configurar parámetros como número de muestras por evento, retraso de muestras y cantidad de eventos a capturar.

Antes de iniciar la adquisición, se verifica el espacio libre en la SD 
y se calcula el número máximo de eventos permitidos, asegurando al menos 
200 MB libres. Los datos adquiridos se almacenan en formato HDF5, 
permitiendo optimización y manipulación eficiente de grandes volúmenes 
de datos. Si se desea trabajar con adquisición de grandes volúmenes de datos
, se puede ejecutar el programa monitor_hdf5 que transmite los datos capturados a través de la red y los borra de la memoria de la RedPitaya para liberar espacio.

La adquisición se detiene si luego de 2 horas, el programa no detecta señales que disparen la adquisición configurada por el usuario.

Nota: El rango de tensión de las entradas de la Red Pitaya depende de la 
configuración del jumper en la placa (HV: ±20 V, LV: ±1 V). Se recomienda 
verificar que las señales medidas estén dentro del rango adecuado antes 
de la adquisición. Para más información, consulte la documentación oficial:
https://redpitaya.readthedocs.io/en/latest/intro.html
#==============================================================

In [ ]:

import os
import time
import numpy as np
import h5py
from datetime import datetime
from zoneinfo import ZoneInfo
import rp
from rp_overlay import overlay
import pathlib
from matplotlib import pyplot as plt

# -----------------------------FUNCIONES-----------------------------------

def get_free_space_mb(path="/"):
    """Obtiene el espacio libre en la SD en MB."""
    statvfs = os.statvfs(path)
    free_space = statvfs.f_bavail * statvfs.f_frsize
    return free_space / (1024 * 1024)


def interpolate_params(channels):
    """Interpola los parámetros F y P (en KB) en función de la cantidad de canales (para 32 muestras)."""
    if channels <= 1:
        return 2.68, 1.59
    elif channels >= 4:
        return 5.10, 2.30
    else:
        F = 2.68 + (5.10 - 2.68) * (channels - 1) / (4 - 1)
        P = 1.59 + (2.30 - 1.59) * (channels - 1) / (4 - 1)
        return F, P


def estimate_file_size(channels, samples, events):
    """Estima el tamaño del archivo (en KB) en función de los eventos."""
    data_payload = (channels * samples * 4) / 1024.0
    F, P = interpolate_params(channels)
    total_size = F + (events - 1) * (data_payload + P)
    return total_size


def get_max_events_or_time(samples, channels):
    """Permite configurar límite por cantidad de eventos o duración."""
    print("\n\n=================================================================")
    print("\033[1m   Configuración de almacenamiento de datos\033[0m")
    print("=================================================================\n")

    free_space = get_free_space_mb()
    available_space_mb = free_space - 200  # Deja 200 MB libres
    if available_space_mb <= 0:
        print("⚠️ No hay suficiente espacio libre en la SD. Libera espacio antes de continuar.")
        exit()

    available_space_kb = available_space_mb * 1024
    per_event_size = estimate_file_size(channels, samples, 1)
    max_events = int(available_space_kb // per_event_size)

    print(f"🔷 Espacio disponible (dejando 200 MB libres): {available_space_mb:.2f} MB")
    print(f"🔷 Máximo de eventos que se pueden guardar: {max_events}")

    print("\n🔴 ¿Deseas limitar la grabación por cantidad de eventos o por duración?")
    print("    E - número de eventos (default)")
    print("    T - duración en minutos")
    option = input(" ").strip().lower()

    if option == 't':
        while True:
            try:
                minutes = float(input("⏳ ¿Cuántos minutos deseas grabar? (Ej: 1.5): ").strip())
                if minutes <= 0:
                    raise ValueError("El tiempo debe ser mayor que cero.")
                print(f"✅ Se grabará durante {minutes} minutos o hasta llenar la SD.")
                return {'mode': 'time', 'duration_minutes': minutes, 'max_events': max_events}
            except ValueError as e:
                print(f"⚠️ Entrada inválida: {e}")
    else:
        default_events = 10
        try:
            user_input = input(f"🔴 ¿Cuántos eventos deseas guardar? (Máximo {max_events}, Enter usa {default_events}): ").strip()
            num_events = int(user_input) if user_input else default_events
        except ValueError:
            print("⚠️ Entrada inválida. Se guardarán 10 eventos por defecto.")
            num_events = default_events

        num_events = min(num_events, max_events)
        est_size = estimate_file_size(channels, samples, num_events)
        print(f"🔷 Tamaño estimado para {num_events} eventos: {est_size:.2f} KB")
        return {'mode': 'events', 'num_events': num_events}


def select_channels(available_channels=[1, 2, 3, 4]):
    """Permite seleccionar qué canales capturar."""
    user_input = input("🔴 Ingresa los canales a capturar (Ej: 1,2,3,4 o 1 2 3 4). 'Enter' usa todos: ")
    if user_input.strip() == "":
        return available_channels
    try:
        channels = [int(ch) for ch in user_input.replace(",", " ").split() if ch.strip().isdigit()]
        channels = [ch for ch in channels if ch in available_channels]
        return channels if channels else available_channels
    except:
        print("⚠️ Error en la entrada, usando todos los canales.")
        return available_channels


def select_trigger_source():
    """Selecciona canal y flanco del trigger."""
    edge_input = input("🔴 Tipo de flanco (P = positivo, N = negativo) [Default: P]: ").strip().upper()
    edge = edge_input if edge_input in ["P", "N"] else "P"

    while True:
        channel_input = input("🔴 Canal de trigger (1–4) [Default: 1]: ").strip()
        try:
            ch = int(channel_input) if channel_input else 1
            if ch in [1, 2, 3, 4]:
                break
            else:
                print("⚠️ Canal inválido. Debe ser 1 a 4.")
        except ValueError:
            print("⚠️ Entrada inválida, usando canal 1.")

    channel_letter = ["A", "B", "C", "D"][ch - 1]
    suffix = "PE" if edge == "P" else "NE"
    const_name = f"RP_TRIG_SRC_CH{channel_letter}_{suffix}"
    trigger_source = getattr(rp, const_name)
    trigger_channel = getattr(rp, f"RP_CH_{ch}")

    print(f"   --> Trigger: Canal {ch}, flanco {'positivo' if edge == 'P' else 'negativo'} ({const_name})")

    return trigger_source, trigger_channel, ch, suffix


def get_user_input(prompt, default_value, cast_type=float):
    """Obtiene un valor del usuario, con valor por defecto."""
    user_input = input(f"{prompt} ('Enter' usa {default_value}): ").strip()
    try:
        return cast_type(user_input) if user_input else default_value
    except ValueError:
        print("⚠️ Entrada inválida, usando valor por defecto.")
        return default_value


def create_custom_time():
    """Permite crear una fecha/hora personalizada."""
    while True:
        try:
            day = int(input("📅 Día (1–31): "))
            month = int(input("📅 Mes (1–12): "))
            year = int(input("📅 Año (ej. 2025): "))
            hour = int(input("🕒 Hora (0–23): "))
            minute = int(input("🕒 Minuto (0–59): "))
            return datetime(year, month, day, hour, minute, 0)
        except ValueError as e:
            print(f"❌ Error: {e}. Intenta de nuevo.")


def generar_nombre_archivo(file_index):
    """Genera nombre de archivo secuencial."""
    return f"Data_{day_time_of_first_pulse}_{file_index:04d}.h5"


# -----------------------------INICIO DEL PROGRAMA-----------------------------------

print("=================================================================")
print("\033[1m          Adquisición de datos para Red Pitaya\033[0m")
print("=================================================================\n")

try:
    os.chdir(os.path.dirname(os.path.abspath(__file__)))
except NameError:
    pass

set_time = datetime.now(ZoneInfo("America/Argentina/Buenos_Aires"))
print(f"\033[1m     Hora del sistema: {set_time}\033[0m\n")
time_option = input(f"🔴 ¿Usar hora del sistema o personalizada? (S = sistema, C = custom): ").strip().lower()
if time_option == 'c':
    set_time = create_custom_time()
    print(f"ℹ️ Hora utilizada: {set_time}\n")

sys_time_ns = int(datetime.now().timestamp() * 1e9)
set_time_ns = int(set_time.timestamp() * 1e9)

print("\n🟢 INICIANDO FPGA...\n")
fpga = overlay()
rp.rp_Init()

dec = rp.RP_DEC_1
trig_dly = 0
N = 16384

print("\n\n=================================================================")
print("\033[1m   Configuración de los parámetros de adquisición\033[0m")
print("=================================================================\n")

acq_trig_sour, trig_channel, channel, flanco = select_trigger_source()
trig_lvl = get_user_input("🔴 Nivel de trigger [V]", 0.01)

rp.rp_AcqSetTriggerSrc(acq_trig_sour)
rp.rp_AcqSetTriggerLevel(trig_channel, trig_lvl)

samples = get_user_input("\n🔴 Número de muestras por evento", 32, int)
samples_delay = get_user_input("🔴 Delay de muestras", 8, int)

available_channels = [1, 2, 3, 4]
channels_to_acquire = select_channels(available_channels)
num_channels = len(channels_to_acquire)
config = get_max_events_or_time(samples, num_channels)
print(config)

fs = 125e6 / dec
dt = 1 / fs
time_axis = np.linspace(0, (samples - 1) * dt, samples)

print("\n\n=================================================================")
print("\033[1m PARÁMETROS DE ADQUISICIÓN SELECCIONADOS\033[0m")
print("=================================================================\n")
print(f"🔷 Frecuencia de muestreo = {fs/1e6:.2f} MHz")
print(f"🔷 Canal de trigger: {channel}")
print(f"🔷 Flanco = {flanco}")
print(f"🔷 Nivel de trigger = {trig_lvl} V")
print(f"🔷 Muestras por evento = {samples}")
print(f"🔷 Delay de muestras = {samples_delay}")
print(f"🔷 Canales seleccionados = {channels_to_acquire}")
print(f"🔷 Hora del sistema: {set_time}")
if config['mode'] == 'events':
    print(f"🔷 Eventos a adquirir: {config['num_events']}.\n")
else:
    print(f"🔷 Adquisición durante {config['duration_minutes']} minutos.\n")

day_time_of_first_pulse = set_time.strftime('%Y%m%d_%H%M')

# --- Crear carpeta de salida ---
BASE_DIR = pathlib.Path("/home/jupyter/RedPitaya/DATOS").resolve()
trig_mV = int(trig_lvl * 1000)
trig_str = f"{trig_mV:+03d}"[:3]
folder_name = f"Data_{day_time_of_first_pulse}_TCH{channel}_TL{trig_str}"
output_dir = BASE_DIR / folder_name
output_dir.mkdir(exist_ok=True, parents=True)
os.chdir(output_dir)
print(f"📂 Ruta a carpeta de DATOS: {output_dir}\n")

# CONFIGURACIÓN DE PARTICIONAMIENTO
file_threshold_bytes = 10 * 1024 * 1024  # 10 MB
current_file_size = 0
file_index = 1
h5file = None

trigger_times = []
event = 0
start_time = time.time()
max_wait_between_triggers = 60
max_total_duration = config['duration_minutes'] * 60 if config['mode'] == 'time' else float('inf')
max_events = config['num_events'] if config['mode'] == 'events' else config['max_events']

print("\033[1m🟢 INICIANDO ADQUISICIÓN...\033[0m\n")

trigger_times = []
first_trigger_ns = None
print("\n=========================================================")
print(f"   Ciclo            Estado            Tiempo relativo  ")
print("=========================================================")
while True:
    elapsed_total = time.time() - start_time
    if config['mode'] == 'events' and event >= max_events:
        break
    if config['mode'] == 'time' and (elapsed_total >= max_total_duration or event >= max_events):
        break

    print(f" {event + 1}/{'∞' if config['mode'] == 'time' else max_events}\t        Esperando   trigger  ")



    rp.rp_AcqStart()
    rp.rp_AcqSetTriggerSrc(acq_trig_sour)
    wait_start = time.time()
    triggered = False

    while True:
        if rp.rp_AcqGetTriggerState()[1] == rp.RP_TRIG_STATE_TRIGGERED:
            triggered = True
            break
        if (time.time() - wait_start) >= max_wait_between_triggers:
            print("No se detectaron triggers durante el tiempo máximo de espera. Finalizando...")
            triggered = False
            break
        if config['mode'] == 'time' and (time.time() - start_time) >= max_total_duration:
            triggered = False
            break
        time.sleep(0.001)
    # print (triggered)
    # print(not(triggered))

    if not triggered:
        rp.rp_AcqStop()
        break

    # # Inicializar el tiempo del trigger relativo a 0 ns para mostrar en pantalla
    # first_trigger_ns = None
    # trigger_times = []
    
    # Tiempo relativo al primer trigger 
    trigger_time_ns = time.time_ns()
    trigger_times.append(trigger_time_ns)
    
    if first_trigger_ns is None:
        first_trigger_ns = trigger_time_ns
        relative_ns = 0
    else:
        relative_ns = trigger_time_ns - first_trigger_ns
    
    # imprimir en ns (o elegir unidad legible si preferís)
    print(f"               → Trigger detectado         {round(relative_ns/1e6)} ms   ")
    print("---------------------------------------------------------")

    if h5file is None:
        current_filename = generar_nombre_archivo(file_index)
        h5file = h5py.File(current_filename, "w")
        # --- metadatos iniciales ---
        h5file.attrs.update({
            'set_time': set_time_ns,
            'sys_time': sys_time_ns,
            'decimation': dec,
            'trigger_level': trig_lvl,
            'trigger_channel': channel,
            'trigger_flank': flanco,
            'trigger_delay': trig_dly,
            'samples_per_event': samples,
            'samples_delay': samples_delay,
            'channels': np.array(channels_to_acquire),
            'sampling_rate': fs,
            'mode': config['mode']
        })
        if config['mode'] == "time":
            h5file.attrs['duration_minutes'] = config['duration_minutes']
        else:
            h5file.attrs['num_events'] = config['num_events']

    time.sleep(samples / (125e6 / dec))

    fbuffers = {ch: rp.fBuffer(N) for ch in channels_to_acquire}
    data = {}
    for ch in channels_to_acquire:
        rp.rp_AcqGetOldestDataV(getattr(rp, f'RP_CH_{ch}'), N, fbuffers[ch])
        data[f'channel_{ch}'] = np.array([
            fbuffers[ch][i + N // 2 - samples_delay] for i in range(samples)
        ], dtype=np.float32)

    # --- Control real de tamaño ---
    if h5file is not None:
        current_file_size = os.path.getsize(h5file.filename)
    event_size_bytes = samples * num_channels * 4

    if h5file is not None and (current_file_size + event_size_bytes >= file_threshold_bytes):
        h5file.close()
        file_index += 1
        current_filename = generar_nombre_archivo(file_index)
        h5file = h5py.File(current_filename, "w")
        h5file.attrs.update({
            'set_time': set_time_ns,
            'sys_time': sys_time_ns,
            'decimation': dec,
            'trigger_level': trig_lvl,
            'trigger_channel': channel,
            'trigger_flank': flanco,
            'trigger_delay': trig_dly,
            'samples_per_event': samples,
            'samples_delay': samples_delay,
            'channels': np.array(channels_to_acquire),
            'sampling_rate': fs,
            'mode': config['mode']
        })
        if config['mode'] == "time":
            h5file.attrs['duration_minutes'] = config['duration_minutes']
        else:
            h5file.attrs['num_events'] = config['num_events']
        print(f"\n📁 Nuevo archivo creado: {current_filename}\n")

    # --- Guardar evento ---
    group = h5file.create_group(f"event_{event + 1:06d}")
    group.attrs['timestamp'] = trigger_time_ns
    for ch in channels_to_acquire:
        group.create_dataset(f"channel_{ch}", data=data[f'channel_{ch}'])

    rp.rp_AcqStop()
    event += 1

# Finalización
if h5file is not None:
    h5file.close()
rp.rp_Release()

print("\n✅ Adquisición finalizada.")

if trigger_times:
    elapsed_time = (trigger_times[-1] - trigger_times[0]) / 1e9
    print(f"\n⏱️ Tiempo entre primer y último trigger: {elapsed_time:.6f} s")



          Adquisición de datos para Red Pitaya

     Hora del sistema: 2025-10-29 17:17:35.171660-03:00



🔴 ¿Usar hora del sistema o personalizada? (S = sistema, C = custom):  s



🟢 INICIANDO FPGA...

Check FPGA [OK].


   Configuración de los parámetros de adquisición



🔴 Tipo de flanco (P = positivo, N = negativo) [Default: P]:  n
🔴 Canal de trigger (1–4) [Default: 1]:  4


   --> Trigger: Canal 4, flanco negativo (RP_TRIG_SRC_CHD_NE)


🔴 Nivel de trigger [V] ('Enter' usa 0.01):  0

🔴 Número de muestras por evento ('Enter' usa 32):  
🔴 Delay de muestras ('Enter' usa 8):  
🔴 Ingresa los canales a capturar (Ej: 1,2,3,4 o 1 2 3 4). 'Enter' usa todos:  




   Configuración de almacenamiento de datos

🔷 Espacio disponible (dejando 200 MB libres): 984.44 MB
🔷 Máximo de eventos que se pueden guardar: 197659

🔴 ¿Deseas limitar la grabación por cantidad de eventos o por duración?
    E - número de eventos (default)
    T - duración en minutos


  
🔴 ¿Cuántos eventos deseas guardar? (Máximo 197659, Enter usa 10):  100000


🔷 Tamaño estimado para 100000 eventos: 280002.30 KB
{'mode': 'events', 'num_events': 100000}


 PARÁMETROS DE ADQUISICIÓN SELECCIONADOS

🔷 Frecuencia de muestreo = 125.00 MHz
🔷 Canal de trigger: 4
🔷 Flanco = NE
🔷 Nivel de trigger = 0.0 V
🔷 Muestras por evento = 32
🔷 Delay de muestras = 8
🔷 Canales seleccionados = [1, 2, 3, 4]
🔷 Hora del sistema: 2025-10-29 17:17:35.171660-03:00
🔷 Eventos a adquirir: 100000.

📂 Ruta a carpeta de DATOS: /home/jupyter/RedPitaya/DATOS/Data_20251029_1717_TCH4_TL+00

🟢 INICIANDO ADQUISICIÓN...


   Ciclo            Estado            Tiempo relativo  
 1/100000	        Esperando   trigger  
               → Trigger detectado         0 ms   
---------------------------------------------------------
 2/100000	        Esperando   trigger  
               → Trigger detectado         61 ms   
---------------------------------------------------------
 3/100000	        Esperando   trigger  
               → Trigger detectado         100 ms   
-----------------------